1. 将收入成本表中的内容按照合并口径->工商名称->合同号的顺序进行依次筛选

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw

## 2021年

In [2]:
hebing = ['中国航天科工集团',
'中国电子科技集团',
'中国航天科技集团',
'中国兵器工业集团',
'杨茂富',
'中国航空工业集团',
'中国船舶集团',
'藏朝峰',
'中国电子信息产业集团',
'江滔',
'王柯',
'丁燕',
'杜春香',
'上海瀚讯',
'王晓燕',
'王洁',
'唐富奎',
'山东省国有资产投资控股',
'宏嘉科技',
'刘英',
'唯力科技',
'康达新材',
'江苏和正',
'赖建明',
'中国兵器装备集团'
]

In [3]:
len(hebing)

25

In [4]:
df = pd.read_excel('./收入成本大表2020-2022.xlsx', sheet_name='2021')
df = df.iloc[:,0:15]

In [5]:
df['合同金额'] = df['数量']*df['销售含税单价']

In [6]:
df = df.reindex(columns = ['合并口径','客户工商名称','合同号','产品代码','数量','销售含税单价','合同金额','出库日期','源单单号','FNote',\
    '日期', '发票号码','凭证号','不含税销售额'])

In [7]:
df.columns.values

array(['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期',
       '源单单号', 'FNote', '日期', '发票号码', '凭证号', '不含税销售额'], dtype=object)

In [8]:
comp = {}  # comp[合并口径]=下属公司 1、客户年总销售量大于20万
for i in hebing:
    temp = df[df['合并口径']==i].groupby('客户工商名称')['不含税销售额'].agg(np.sum).sort_values()
    comp_name = temp[temp>=200000].index
    comp[i] =comp_name

In [9]:
n = 0
for i in range(len(hebing)):
    a = len(comp[hebing[i]])
    n += a    
n

59

In [10]:
comp[hebing[0]]

Index(['北京遥感设备研究所', '湖北楚航电子科技有限公司', '中国航天科工防御技术研究院物资供应站', '北京飞航捷迅物资有限责任公司',
       '贵州航天林泉电机有限公司', '青岛航天半导体研究所有限公司', '北京计算机技术及应用研究所', '北京无线电测量研究所',
       '北京华航海鹰新技术开发有限责任公司', '中国长峰机电技术研究设计院'],
      dtype='object', name='客户工商名称')

In [11]:
contracts ={}  # contracts[公司名称]=合同编号 2、抽取金额最大的三个合同，
for i in hebing:
    for j in comp[i]:
        temp = df[(df['客户工商名称']==j) & (df['合并口径']==i)].groupby('合同号')['不含税销售额'].agg(np.sum).sort_values(ascending=False)
        if len(temp)<=3:
            contracts[j] = temp.index
        else:
            contracts[j] = temp.index[0:3]

In [12]:
contracts['贵州航天林泉电机有限公司']

Index(['DP0720210416-06', 'DP0720210304', 'DP0720210617-02'], dtype='object', name='合同号')

In [13]:
out = pd.DataFrame()
for i in hebing: # 集团
    for j in comp[i]: # 公司
        for c_id in contracts[j]: # 合同
            temp = df[(df['合同号']==c_id) & (df['合并口径']==i) & (df['客户工商名称']==j)]           
            out = pd.concat([out, temp])

In [14]:
# out.to_excel('test.xlsx', encoding='utf-8', index=False)

In [15]:
wb = xw.Book('2-2-7.xlsx')

In [16]:
sht1 = wb.sheets[0]

In [117]:
sht1['A5:AJ657'].clear_contents()

In [24]:
sht1['B5'].value = out.iloc[:,0:9].values

In [18]:
out.columns.values

array(['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期',
       '源单单号', 'FNote', '日期', '发票号码', '凭证号', '不含税销售额'], dtype=object)

In [152]:
sht1['M5:M3982'].api.NumberFormat = "@"

In [25]:
sht1['M5'].options(transpose=True).value = out['FNote'].values

In [149]:
out['FNote'].values

array(['SF1099745648671', 'SF1099745648671', 'SF1099745648671', ...,
       'SF1121099126045', 'SF1121099126045', 'SF1121099126045'],
      dtype=object)

In [150]:
sht1['M5'].value = out['FNote'].values

In [155]:
sht1['AD5:AD3982'].api.NumberFormat = "@"

In [26]:
sht1['AD5'].options(transpose=True).value = out['发票号码'].values

In [168]:
sht1['AC5:AC3982'].api.NumberFormat = "yyyy/mm/dd"

In [27]:
sht1['AC5'].options(transpose=True).value = out['日期'].values

In [28]:
sht1['AF5'].options(transpose=True).value = out['不含税销售额'].values

In [29]:
sht1['AG5'].options(transpose=True).value = out['凭证号'].values

In [17]:
# 合并
out.columns

Index(['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期', '源单单号',
       'FNote', '日期', '发票号码', '凭证号', '不含税销售额'],
      dtype='object')

In [18]:
# 合并东方控制表 (主键：客户名称、合同号、产品代码、出库单号、数量、快递单号)这些信息可以标记一笔收入确认
dongfang = pd.read_excel('./控制表-收入细节性测试-已脱密.xlsx', sheet_name=1, skiprows=2, usecols=[i for i in range(2, 35)])
dongfang.columns.values

array(['合并工商名称', '底稿对应情况（已脱敏底稿以M后缀命名）', '收入凭证年月', '凭证号', '收入金额', '合同号',
       '标的物型号', '出库时间', '出库单号', '数量', '发票号码', '开票日期', '数量.1', '不含税金额',
       '客户签收确认时间', '合同号.1', '出库单号.1', '快递单号', '发件时间', '回款凭证日期', '回款凭证号',
       '客户名称/客户代码.1', '对方科目', '回款金额', '汇款日期', '汇款方名称/汇款方代码', '是否第三方回款',
       '金额', '票据类型', '票号', '出票人/出票人代码', '承兑人/承兑人代码', '金额.1'], dtype=object)

In [19]:
# 改名
out.rename(columns={'源单单号':'出库单号', 'FNote':'快递单号', '日期':'开票日期', '凭证号':'收入凭证号'}, inplace=True)
dongfang.rename(columns={'合并工商名称':'客户工商名称', '标的物型号':'产品代码'}, inplace=True)

In [22]:
new = out.merge(dongfang, how='left', on=['客户工商名称','合同号','产品代码','数量','出库单号'])
new.columns

Index(['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期', '出库单号',
       '快递单号_x', '开票日期_x', '发票号码_x', '收入凭证号', '不含税销售额', '底稿对应情况（已脱敏底稿以M后缀命名）',
       '收入凭证年月', '凭证号', '收入金额', '出库时间', '发票号码_y', '开票日期_y', '数量.1', '不含税金额',
       '客户签收确认时间', '合同号.1', '出库单号.1', '快递单号_y', '发件时间', '回款凭证日期', '回款凭证号',
       '客户名称/客户代码.1', '对方科目', '回款金额', '汇款日期', '汇款方名称/汇款方代码', '是否第三方回款', '金额',
       '票据类型', '票号', '出票人/出票人代码', '承兑人/承兑人代码', '金额.1'],
      dtype='object')

In [ ]:
# 货款结算方式==回款凭证号+/+'对方科目'+'票据类型'
# 进账日期==回款凭证日期
# 金额==汇款金额
# 对方名称=='汇款方名称/汇款方代码'+'出票人/出票人代码'

In [24]:
new['出库数量'] = new['数量']
new['快递数量'] = new['数量']
new['客户签收数量'] = new['数量']
new['发票数量']=new['数量']
new['记账数量'] = new['数量']
new['货款结算方式'] = new['回款凭证号']+'/'+new['对方科目']+'/'+new['票据类型']

In [28]:
new['快递单号'] = new.apply(lambda x:x['快递单号_x'] if x['快递单号_x']==x['快递单号_y'] or pd.isnull(x['快递单号_y']) else x['快递单号_y'],axis=1)

In [40]:
new['运输单'] = new.apply(lambda x: '快递单' if x['快递单号'][-1].isdigit() else x['快递单号'], axis=1)

C:\Users\wo\AppData\Local\Temp\ipykernel_6184\1412738457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['运输单'] = new.apply(lambda x: '快递单' if x['快递单号'][-1].isdigit() else x['快递单号'], axis=1)


In [33]:
new['签收单'] = '交付确认单'

In [34]:
new['记账凭证号'] = new['收入凭证号']+'/'+new['凭证号']

In [35]:
new['对方名称'] = new['汇款方名称/汇款方代码']+new['承兑人/承兑人代码']

In [32]:
new.columns

Index(['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', '出库日期', '出库单号',
       '快递单号_x', '开票日期_x', '发票号码_x', '收入凭证号', '不含税销售额', '底稿对应情况（已脱敏底稿以M后缀命名）',
       '收入凭证年月', '凭证号', '收入金额', '出库时间', '发票号码_y', '开票日期_y', '数量.1', '不含税金额',
       '客户签收确认时间', '合同号.1', '出库单号.1', '快递单号_y', '发件时间', '回款凭证日期', '回款凭证号',
       '客户名称/客户代码.1', '对方科目', '回款金额', '汇款日期', '汇款方名称/汇款方代码', '是否第三方回款', '金额',
       '票据类型', '票号', '出票人/出票人代码', '承兑人/承兑人代码', '金额.1', '出库数量', '快递数量',
       '客户签收数量', '发票数量', '记账数量', '货款结算方式', '进账日期', '快递单号', '运输单'],
      dtype='object')

In [41]:
new = new[['合并口径', '客户工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额', \
    '出库日期', '出库单号','出库数量',\
    '运输单', '快递单号', '发件时间', '快递数量',\
    '签收单', '客户签收数量', '客户签收确认时间', '客户工商名称',\
    '开票日期_x', '发票号码_x', '发票数量', '不含税销售额',\
    '记账凭证号', '收入凭证年月', '不含税销售额', '记账数量',\
    '货款结算方式', '回款凭证日期', '回款金额',  '对方名称']]

In [42]:
new.to_excel('new.xlsx', encoding='utf-8')

In [128]:
sht1['A5:AJ887'].clear_contents()

In [129]:
sht1['B5'].value = new.iloc[:,0:9].values

In [132]:
new.isnull().sum()

合并口径                     0
客户工商名称                   0
合同号                      0
产品代码                     0
数量                       0
销售含税单价                   0
合同金额                     0
出库日期                     0
出库单号                     0
快递单号_x                   0
开票日期_x                   0
发票号码_x                   0
记账凭证号_x                  0
不含税销售额                   0
底稿对应情况（已脱敏底稿以M后缀命名）    771
收入凭证年月                 771
记账凭证号_y                771
收入金额                   771
出库时间                   771
发票号码_y                 771
开票日期_y                 771
数量.1                   771
不含税金额                  771
客户签收确认时间               771
合同号.1                  771
出库单号.1                 771
快递单号_y                 771
发件时间                   774
回款凭证日期                 825
回款凭证号                  771
客户名称/客户代码.1            826
对方科目                   825
回款金额                   825
汇款日期                   867
汇款方名称/汇款方代码            867
是否第三方回款                867
金额                     867
票

In [139]:
sht1['M5'].options(transpose=True).value = new['快递单号_x'].values
# sht1['N5'].options(transpose=True).value = new['发件时间'].values
# sht1['AA5:AA887'].api.NumberFormat = "yyyy-mm-dd;"
# sht1['AA5'].options(transpose=True).value = new['客户签收确认时间'].values
# sht1['AC5:AC887'].api.NumberFormat = "yyyy/m/d;"
# sht1['AC5'].options(transpose=True).value = new['开票日期_x'].values
sht1['AD5'].options(transpose=True).value = new['发票号码_x'].values
sht1['AF5'].options(transpose=True).value = new['不含税销售额'].values
sht1['AG5'].options(transpose=True).value = new['记账凭证号_y'].values
# sht1['AH5:AH887'].api.NumberFormat = "yyyy/m/d;"
# sht1['AH5'].options(transpose=True).value = new['收入凭证年月'].values
sht1['AL5'].options(transpose=True).value = new['回款凭证号'].values
# sht1['AM5:AM887'].api.NumberFormat = "yyyy/m/d;"
# sht1['AM5'].options(transpose=True).value = new['回款凭证日期'].values
sht1['AN5'].options(transpose=True).value = new['回款金额'].values

In [142]:
sht1['AT5'].options(transpose=True).value = new['底稿对应情况（已脱敏底稿以M后缀命名）'].values

In [140]:
new.dtypes

合并口径                           object
客户工商名称                         object
合同号                            object
产品代码                           object
数量                              int64
销售含税单价                        float64
合同金额                          float64
出库日期                   datetime64[ns]
出库单号                           object
快递单号_x                         object
开票日期_x                 datetime64[ns]
发票号码_x                         object
记账凭证号_x                        object
不含税销售额                        float64
底稿对应情况（已脱敏底稿以M后缀命名）            object
收入凭证年月                         object
记账凭证号_y                        object
收入金额                          float64
出库时间                   datetime64[ns]
发票号码_y                         object
开票日期_y                 datetime64[ns]
数量.1                          float64
不含税金额                         float64
客户签收确认时间               datetime64[ns]
合同号.1                          object
出库单号.1                         object
快递单号_y      

In [133]:
# 判断快递单号是否存在差异
new.to_excel('new.xlsx', encoding='utf-8')

In [ ]:
# 2020年